In [135]:
"""
title: CRISPR Analysis Program of E. amylovora 
author: Memeti Nurdzane
date: June 2021
mail: memetnur@students.zhaw.ch
supervisor: theo smits

"""
#Import packages and data
import pandas as pd
import numpy as np
from Bio import pairwise2
from Bio.Seq import Seq
from Bio.pairwise2 import format_alignment

!pip install biopython

# create a dataframe (df) where we import data from excel. note that no entry is labelled here as "NaN". 
df = pd.read_excel('Rubus_CRISPRs_sequence_analysis.xlsx', 'CRR')

In [136]:
df 

,Ea02_03_NZ_JAAEWG010000099,Ea03_03_NZ_JAAEWF010000099,Ea1_95_NZ_JAAEVV010000099,Ea6_96_NZ_JAAEVB010000099,Ea8_96_NZ_JAAEUY010000099_reversed,Ea04_03_NZ_JAAEWE010000099,Ea1_98_NZ_JAAEVT010000099,Ea7_96_NZ_JAAEUZ010000099_4bases deleted,Ea2_95_NZ_JAAEVS010000009,EaLevo2_NZ_JAAEUQ010000009,Ea1_97_NZ_JAAEVU010000099,Ea2_97_NZ_JAAEVQ010000099,Ea3_97_NZ_JAAEVM010000099,Ea4_96_NZ_JAAEVG010000099_reverse_complement,Ea644_NZ_CAPD00000000.1,MR1_NZ_CAPE00000000
0,>spacer1,>spacer1,>spacer1,>spacer1,>spacer1,>spacer1,>spacer1,>spacer1,>spacer1,>spacer1,>spacer1,>spacer1,>spacer1,>spacer1,>spacer1,>spacer1
1,TTACGTTTGCGTTAACAGTAAGCTCTGCAAC,TTACGTTTGCGTTAACAGTAAGCTCTGCAAC,CCAGAGGACAACTTTCCGCCTTTTCTCTGCGT,GAATTTACGTTTGCGTTAACAGTAAGCTCTGCAAC,ATCGCACCCCACTGATTGAAGAGCAGCACACT,GAATTTACGTTTGCGTTAACAGTAAGCTCTGCAAC,TTACGTTTGCGTTAACAGTAAGCTCTGCAAC,TTACGTTTGCGTTAACAGTAAGCTCTGCAAC,GAATTTACGTTTGCGTTAACAGTAAGCTCTGCAAC,TTACGTTTGCGTTAACAGTAAGCTCTGCAAC,TTACGTTTGCGTTAACAGTAAGCTCTGCAAC,GAATTTACGTTTGCGTTAACAGTAAGCTCTGCAAC,TTACGTTTGCGTTAACAGTAAGCTCTGCAAC,TTACGTTTGCGTTAACAGTAAGCTCTGCAAC,TTACGTTTGCGTTAACAGTAAGCTCTGCAAC,TTACGTTTGCGTTAACAGTAAGCTCTGCAAC
2,>spacer2,>spacer2,>spacer2,>spacer2,>spacer2,>spacer2,>spacer2,>spacer2,>spacer2,>spacer2,>spacer2,>spacer2,>spacer2,>spacer2,>spacer2,>spacer2
3,GGCTAAATGCCTGAATTCAACGTTCTACAAAA,GGCTAAATGCCTGAATTCAACGTTCTACAAAA,AGGTGGCGGAATACGGCAACGAGTGGATAACT,GAAAGGCTAAATGCCTGAATTCAACGTTCTACAAAA,GAGCAATATATTCGATAGACGCTGATTTGCGT,GAAAGGCTAAATGCCTGAATTCAACGTTCTACAAAA,GGCTAAATGCCTGAATTCAACGTTCTACAAAA,GGCTAAATGCCTGAATTCAACGTTCTACAAAA,GAAAGGCTAAATGCCTGAATTCAACGTTCTACAAAA,GGCTAAATGCCTGAATTCAACGTTCTACAAAA,GAGCAATATATTCGATAGACGCTGATTTGCGT,GAAAGGCTAAATGCCTGAATTCAACGTTCTACAAAA,GGCTAAATGCCTGAATTCAACGTTCTACAAAA,GGCTAAATGCCTGAATTCAACGTTCTACAAAA,GGCTAAATGCCTGAATTCAACGTTCTACAAAA,GGCTAAATGCCTGAATTCAACGTTCTACAAAA
4,>spacer3,>spacer3,>spacer3,>spacer3,>spacer3,>spacer3,>spacer3,>spacer3,>spacer3,>spacer3,>spacer3,>spacer3,>spacer3,>spacer3,>spacer3,>spacer3
5,GAAAATCGCACCCCACTGATTGAAGAGCAGCACACT,ATCGCACCCCACTGATTGAAGAGCAGCACACT,CGGATCACCTCTTCGCCCAGCCAGCTGTTCAG,GAAAATCGCACCCCACTGATTGAAGAGCAGCACACT,TGGCTTCAATTGCGGTCGGGTGTGATGCATCA,GAAAATCGCACCCCACTGATTGAAGAGCAGCACACT,ATCGCACCCCACTGATTGAAGAGCAGCACACT,ATCGCACCCCACTGATTGAAGAGCAGCACACT,GAAAATCGCACCCCACTGATTGAAGAGCAGCACACT,ATCGCACCCCACTGATTGAAGAGCAGCACACT,TGGCTTCAATTGCGGTCGGGTGTGATGCATCA,GAAAATCGCACCCCACTGATTGAAGAGCAGCACACT,ATCGCACCCCACTGATTGAAGAGCAGCACACT,ATCGCACCCCACTGATTGAAGAGCAGCACACT,ATCGCACCCCACTGATTGAAGAGCAGCACACT,ATCGCACCCCACTGATTGAAGAGCAGCACACT
6,>spacer4,>spacer4,>spacer4,>spacer4,>spacer4,>spacer4,>spacer4,>spacer4,>spacer4,>spacer4,>spacer4,>spacer4,>spacer4,>spacer4,>spacer4,>spacer4
7,TAAAGAGCAATATATTCGATAGACGCTGATTTGCGT,GAGCAATATATTCGATAGACGCTGATTTGCGT,GAGCAATATATTCGATAGACGCTGATTTGCGT,TAAAGAGCAATATATTCGATAGACGCTGATTTGCGT,GCCAACGTTCACTGTCATTTAGCCACGCTTCGG,TAAAGAGCAATATATTCGATAGACGCTGATTTGCGT,GAGCAATATATTCGATAGACGCTGATTTGCGT,GAGCAATATATTCGATAGACGCTGATTTGCGT,TAAAGAGCAATATATTCGATAGACGCTGATTTGCGT,GAGCAATATATTCGATAGACGCTGATTTGCGT,GCCAACGTTCACTGTCATTTAGCCACGCTTCGG,TAAAGAGCAATATATTCGATAGACGCTGATTTGCGT,GAGCAATATATTCGATAGACGCTGATTTGCGT,GAGCAATATATTCGATAGACGCTGATTTGCGT,GAGCAATATATTCGATAGACGCTGATTTGCGT,GAGCAATATATTCGATAGACGCTGATTTGCGT
8,>spacer5,>spacer5,>spacer5,>spacer5,NaN,>spacer5,>spacer5,>spacer5,>spacer5,>spacer5,NaN,>spacer5,>spacer5,>spacer5,>spacer5,>spacer5
9,GAAATGGCTTCAATTGCGGTCGGGTGTGATGCATCA,TGGCTTCAATTGCGGTCGGGTGTGATGCATCA,TGGCTTCAATTGCGGTCGGGTGTGATGCATCA,GAAATGGCTTCAATTGCGGTCGGGTGTGATGCATCA,NaN,GAAATGGCTTCAATTGCGGTCGGGTGTGATGCATCA,TGGCTTCAATTGCGGTCGGGTGTGATGCATCA,TGGCTTCAATTGCGGTCGGGTGTGATGCATCA,GAAATGGCTTCAATTGCGGTCGGGTGTGATGCATCA,TGGCTTCAATTGCGGTCGGGTGTGATGCATCA,NaN,GAAATGGCTTCAATTGCGGTCGGGTGTGATGCATCA,TGGCTTCAATTGCGGTCGGGTGTGATGCATCA,TGGCTTCAATTGCGGTCGGGTGTGATGCATCA,TGGCTTCAATTGCGGTCGGGTGTGATGCATCA,TGGCTTCAATTGCGGTCGGGTGTGATGCATCA


In [137]:
### Create a dictionary, where strains are stored in key format and spacers stored in values. Note that dictionary elements are accessed via keys, not their position/order. 

dic = {}
keys = list(df.columns) # keys variables stored as strain names and made a list form of it due data format of "df.columns" is in pandas.core.indexes.base.Index
values = []
#len(keys) #check up how many strains?


In [138]:
### Spacers are stored from Dataframe (=df.loc) into a list (=values) and put into the dictionary "dic". Now we have our first dictionary with strains as keys and spacers as values stored.
def check_begin(x):
    # check the len of split [0] 
    # for the bases AAA with 4 Nucleo acid
    if len(x[0:4].split('AAA')[0]) == 1:
        # strip() removes white spaces in begining and end
        return x[4:].strip()
   
    return x.strip()

for key in keys: #key = a loop through all strains
    temp = []

    for val in df.loc[:,key]: #variables val stands for the line entries, note here that datatype of  val is string and float. Float object has no attribute "startswith"
        try:
            if val.startswith('>spacer'): #error: 'float' object has no attribute 'startswith'
                None
            elif (val.startswith('A') | val.startswith('G') | val.startswith('C') | val.startswith('T')):
                temp.append(check_begin(val))
            else:
                None
        except:
            pass
    values.append(temp)
#chek up: for item in temp:

for key in keys:
    dic[key] = values.pop(0) #pop() is an inbuilt function in Python that removes and returns last value from the list or the given index value.
#dic

#fill the gaps:

for k, v in dic.items():
    if k.startswith('Ea1_97'):
        dic[k].insert(1, 'None')
        dic[k].insert(2, 'None')
    elif k.startswith('Ea8_96'):
        dic[k].insert(0, 'None')
        dic[k].insert(1, 'None')

In [139]:
### create an unique list of spacers, which appear just once in the whole dictionary:

unique = []
unique1 = []
for d in dic.values():
    unique.append(d)


#unique = list(set(list(np.array(unique).flatten())))
#for i in unique:
#    print(len(i))
for i in list(np.array(unique).flatten()): #here problems with ndarrays with different lengths or shapes
    if i not in unique1:
        unique1.append(i)

unique1.remove("None")
unique1

['TTACGTTTGCGTTAACAGTAAGCTCTGCAAC',
 'GGCTAAATGCCTGAATTCAACGTTCTACAAAA',
 'ATCGCACCCCACTGATTGAAGAGCAGCACACT',
 'GAGCAATATATTCGATAGACGCTGATTTGCGT',
 'TGGCTTCAATTGCGGTCGGGTGTGATGCATCA',
 'GCCAACGTTCACTGTCATTTAGCCACGCTTCGG',
 'CCAGAGGACAACTTTCCGCCTTTTCTCTGCGT',
 'AGGTGGCGGAATACGGCAACGAGTGGATAACT',
 'CGGATCACCTCTTCGCCCAGCCAGCTGTTCAG',
 'GAATTTACGTTTGCGTTAACAGTAAGCTCTGCAAC']

In [140]:
### Comparison

seq2 = Seq("AGGTGGCGGAATACGGCAACGAGTGGATAACT")
seq1 = Seq("AGGTGGCGGAATACGGCAACGAGTGGATAACT")
# list of seq to remove
remove = []

# list to replace contains the seq which replace the removed seq
replace = []


def cal_score(x, y):
    seq1 = x
    seq2 = y
    # Finding similarities
    alignments = pairwise2.align.globalxx(seq1, seq2)
    scores = 0 
    # Showing results
    for alignment in alignments:
    #print(type(alignment))
    #print(alignment.score)
    #print(format_alignment(*alignment))
        scores = alignment.score + scores
    #print(len(alignments))
        #print(format_alignment(*alignment))
    #print(seq1, len(seq1))
    #print(seq2, len(seq2))

    score = scores/len(alignments)

    #print(scores/len(alignments))

    if ((len(seq1) != len(seq2)) & (score == (min(len(seq1), len(seq2))))):
        print('match')
        print(seq1, len(seq1))
        print(seq2, len(seq2))
        print(score)
        
        remove.append(seq1 if len(seq1) > len(seq2) else seq2)
        replace.append(seq2 if len(seq1) > len(seq2) else seq1)
        # remove the larger one when they are of different length
        unique1.remove(seq1 if len(seq1) > len(seq2) else seq2)


    elif (len(seq1) == len(seq2)) & (score >= (len(seq1) - 4)):
        print('match')
        print(seq1, len(seq1))
        print(seq2, len(seq2))
        print(score)
        unique1.remove(seq1)
    else:
        None
        #print('nomatch')
        
cal_score(seq1,seq2)


match
AGGTGGCGGAATACGGCAACGAGTGGATAACT 32
AGGTGGCGGAATACGGCAACGAGTGGATAACT 32
32.0


In [141]:
### Compare if the spacers in the unique list are not same, rendundant. 
for i in range(len(unique1)):
    for j in unique1[i + 1:]:
        (cal_score(unique1[i], j))

match
TTACGTTTGCGTTAACAGTAAGCTCTGCAAC 31
GAATTTACGTTTGCGTTAACAGTAAGCTCTGCAAC 35
31.0


In [142]:
# this indicates the seq removed
# we keep track to replace this in dic file with 
remove
replace

['TTACGTTTGCGTTAACAGTAAGCTCTGCAAC']

In [143]:
#replace the spacers
for key, val in dic.items():
    for index, item in enumerate(remove):
        for n, i in enumerate(val):
            if i == item:
                val[n] = replace[index]
    dic[key] = val


In [149]:
keys = list(dic.keys())
df = pd.DataFrame(columns=['Identifier', 'Unique'] + keys)
df.Unique = unique1
df.Identifier= df.index+3001
df = df.fillna('')
df.index=np.arange(1,len(df)+1)

for col in df.columns[2:]: #column 2 starts with the strains
    match_position= []
    for j in range(len(dic[col])):
        if dic[col][j] in unique1:
            match_position.append(unique1.index(dic[col][j]) + 1)
    df.loc[match_position,col] = 1

keys = []
for i in list(df.columns):
    keys.append(i.split('_NZ')[0]) #abkürzung

df.columns = keys
df.to_excel('CRISPR_Cluster_Analysis_CRR3_Rubus.xlsx')